In [193]:
import pandas as pd
import numpy as np
import datetime
import re
import requests

In [169]:
KP_API_KEY = '6D9ZG1R-5ATMRD5-P9VZN9W-NP13D77'

In [133]:
interactions = pd.read_parquet('interactions.parquet',engine='pyarrow')
movies_md = pd.read_parquet('movies_metdata.parquet',engine='pyarrow')

In [4]:
interactions.head(5)

,year,month,day,user_id,movie_id,watch_duration_minutes
0,2022,10,1,58073,a6889772-f1f4-45bc-9663-85b46fc8499d,4.800000
1,2022,10,1,63698,c829f262-ddf1-46b7-a896-a7efc205028c,15.000000
2,2022,10,1,66655,91f9d892-a508-4962-91e9-abacd93e0830,51.916668
3,2022,10,1,66655,58b805e1-a65c-49f4-b302-865cacaaed8a,22.316668
4,2022,10,1,67981,dbc44c07-46e2-4fb0-b57c-8b5172421683,117.000000


In [35]:
movies_md.head(5)

,movie_id,title,entity_type,genres,actors,director,country,release_world,age_rating,duration,age,user_id,popularity
0,395a85a9-2200-4a29-a878-97753c471f79,Свиридовы,Серия,"[""Мелодрамы""]","[""Андрей Мерзликин"",""Алексей Горбунов"",""Елена ...","[""Эльдар Салаватов""]","[""Россия""]",2013-12-15,16.0,49.0,113.0,NaN,NaN
1,6fd5f7b5-8278-4d9a-b03a-54a8bf0bc7c8,Охотники за головами,Серия,"[""Детективы"",""Драмы""]","[""Филипп Янковский"",""Елизавета Боярская"",""Конс...","[""Иван Шурховецкий""]","[""Россия""]",2014-03-03,16.0,48.0,110.0,NaN,NaN
2,b239c99e-d1ea-4c01-b83f-6b5469a6b70a,Молодое поколение,Серия,"[""Комедии"",""Мелодрамы""]","[""Юн Пак"",""Пак Ын-бин"",""Хан Е-ри"",""Ким Мин-сок...","[""Ким Сан-хо"",""Ли Тхэ-гон""]","[""Корея Южная""]",2016-07-22,16.0,65.0,82.0,NaN,NaN
3,9dc02ab1-7bd2-45c2-9b8c-ae29e9813ccf,Художницы,Серия,"[""Документальное""]","[""Катарина Лопаткина""]","[""Школа Masters""]","[""Россия""]",2019-02-01,18.0,27.0,51.0,NaN,NaN
4,54b3b5fe-7d2a-47ba-9a1d-4dc3a3a8ff96,Веселая Астрология,Серия,"[""Мультфильмы""]","[""Стивен Боунс"",""Хашир Хайзал Хаилми"",""Логандр...","[""Кен Фунг""]","[""Малайзия""]",2017-10-01,6.0,3.0,67.0,NaN,NaN


# Adding features

In [10]:
movies_md.dtypes

movie_id          object
title             object
entity_type       object
genres            object
actors            object
director          object
country           object
release_world     object
age_rating       float64
duration         float64
dtype: object

In [134]:
# Age: today - release
today = '2023-05-10'

movies_md['release_world']= pd.to_datetime(movies_md['release_world'])
for_age = pd.to_datetime(today)

age_list = []

for date in movies_md['release_world']:
    years = for_age.year - date.year
    months = for_age.month - date.month
    age = years * 12 + months
    
    age_list.append(age)
    
movies_md['age'] = age_list
    

In [135]:
# Popularity: number of watches per film

popularity = interactions.groupby('movie_id')['user_id'].count().reset_index()
popularity.columns = ['movie_id', 'popularity']

movies_md = movies_md.join(popularity.set_index('movie_id'), how='left', on='movie_id')
movies_md['popularity'] = movies_md['popularity'].fillna(0)

In [136]:
# AVG time: average time of watching the film / duration of the film

avg_time_min = interactions.groupby('movie_id')['watch_duration_minutes'].mean().reset_index()
avg_time_min = avg_time_min.join(movies_md[['movie_id', 'duration']].set_index('movie_id'), how='left', on='movie_id')
avg_time_min['avg_time'] = avg_time_min['watch_duration_minutes'] / avg_time_min['duration'] * 100
avg_time_min = avg_time_min[['movie_id', 'avg_time']]

movies_md = movies_md.join(avg_time_min.set_index('movie_id'), how='left', on='movie_id')
movies_md['avg_time'] = movies_md['avg_time'].fillna(0)

In [137]:
# Genre popularity: how popular the genre is overall

def genres_to_list(x):
    if x is not None:
        x = x.replace(',', ',')
        x = x.replace('"', '')
        x = x.replace('[', '')
        x = x.replace(']', '')
        x = x.split(',')
        return x
    return x


movies_md['genres'] = movies_md['genres'].apply(lambda x: genres_to_list(x))

In [ ]:
for_genres = interactions[['user_id', 'movie_id']].copy()
for_genres = for_genres.join(movies_md[['movie_id', 'genres']].set_index('movie_id'), how='left', on='movie_id')

# count every genre in description
# genres_all_inter = [item for sublist in for_genres.genres.dropna() for item in sublist] 

# count top genre in description


# Get rating and description

In [222]:
name = 'Азбука финансовой грамотности'
headers = {'X-API-KEY': KP_API_KEY}

response = requests.get('https://api.kinopoisk.dev/v1.3/movie', params={'selectFields':['rating.kp','description'],'name':name,'limit':1}, headers=headers)
rating = response.json()
len(rating['docs'])

0

In [235]:
import requests
headers = {'X-API-KEY': KP_API_KEY}

def get_rating(name):
    response = requests.get('https://api.kinopoisk.dev/v1.3/movie', params={'selectFields':['rating.kp','description'],'name':name,'limit':1}, headers=headers)
    rating = response.json()
    if len(rating['docs']) != 0:
        rating = pd.DataFrame(rating['docs'])
        rating['rating'] = [dict(x).values() for x in rating['rating']]
        rating['name'] = name
        
        return rating
    
    else:
        return pd.DataFrame({'rating':-1, 'description':' ', 'name':name}, index=[0])

a = get_rating('Азбука финансовой грамотности')
a

,rating,description,name
0,-1,,Азбука финансовой грамотности


In [255]:
interactions_new = interactions.join(movies_md[['movie_id', 'title']].set_index('movie_id'), how='left', on='movie_id')

In [254]:
ratings_n_descrs = pd.DataFrame()

for name in interactions_new['title'].unique().tolist():
    ratings_n_descrs = pd.concat([ratings_n_descrs, get_rating(name)])

In [256]:
ratings_n_descrs.to_csv('ratings_n_descrs.csv')

,rating,description,name
0,(6.664),"Мама, папа и… пятеро щенят. У Барбоскиных все,...",Барбоскины
0,(6.383),"Эми — амбициозная девушка, всегда мечтавшая о ...",Эскортницы
0,(6.936),"Вадик, молодой опер из Москвы, переводится в П...",1703
0,(7.167),Две молодые американки Вики и Кристина проводя...,КрисТина
0,(7.387),Семь лет назад Джейк влип в неприятную историю...,Револьвер


# DL

In [ ]:
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [1]:
import pandas as pd

In [2]:
import tensorflow as tf

: 

: 

In [45]:
interactions[interactions['watch_duration_minutes'] == interactions['watch_duration_minutes'].max()]

,year,month,day,user_id,movie_id,watch_duration_minutes
938078,2022,11,17,228424759,b2cd3955-e7a7-4ba4-b9ef-b3f86df35de2,1755.366699


In [53]:
movies_md[movies_md['movie_id'] == 'fff15902-2da2-4daf-854d-054bf38ece7c']

,movie_id,title,entity_type,genres,actors,director,country,release_world,age_rating,duration,age,popularity
114602,fff15902-2da2-4daf-854d-054bf38ece7c,Движение вперед,Фильм,"[""Документальное""]","[""Бенедикт Бём""]","[""Дарио Тубальдо""]","[""Франция""]",2020-02-01,16.0,21.0,39.0,18.0
